In [69]:
# The code was removed by DSX for sharing.

0     After waiting what seemed like  G52   forever...
1     Yeah   G15   it was such a change I  G40   sw...
2       G13  And I was so glad that all of the meas...
3     Ben and I had been saying for the past two we...
4     Of course   G20  that is what happened for th...
Name: sentence, dtype: object

In [70]:

class data():
    def __init__(self):
        self.data = []
        
class mini_data():
    def __init__(self):
        self.mini_data = []
        
d = data()

'''separate words in sentences and store them in lists'''
def read_data(sentence):
    md = mini_data()
    sentence = sentence.lower()
    content = sentence.split()
    new = md.mini_data + content
    d.data.append(new)

df_data['sentence'].apply(read_data)
training_data = np.array(d.data)
print(training_data[38])

print('training_data length: ' + str(len(training_data)))

avrgs = [len(x) for x in training_data]
print('longest sentence: ' + str(np.max(avrgs)))
print('shortest sentence: ' + str(np.min(avrgs)))
print('avrg sentence length: ' + str(np.mean(avrgs)))
print('standard deviation of sentence length: ' + str(np.std(avrgs)))

'''sentence length will be the average length plus the standard deviation'''
sentence_length = round(np.mean(avrgs) + np.std(avrgs))
print('sentence length: ' + str(sentence_length))


['ever', 'since', 'we', 'were', 'g33', 'g5', 'five', 'our', 'dads', 'take', 'us', 'camping', 'and', 'let', 'me', 'tell', 'yall', 'whose', 'tent', 'was', 'the', 'brightest', 'g11', 'tent', 'ever', 'thats', 'right', 'mine', 'it', 'even', 'out', 'shone', 'henrys', 'but', 'g34', 'thats', 'so', 'you', 'know', 'wed', 'have', 'a', 'point', 'of', 'g6', 'reference', 'if', 'any', 'of', 'us', 'got', 'lost', 'in', 'the', 'night', 'times', 'when', 'there', 'are', 'scary', 'things', 'around', 'and', 'the', 'portapotty', 'seems', 'like', 'a', 'million', 'g56', 'miles', 'away']
training_data length: 874
longest sentence: 150
shortest sentence: 1
avrg sentence length: 27.0778032037
standard deviation of sentence length: 16.6001126717
sentence length: 44.0


In [71]:
import collections

def filter_gestures(word):
    ges = re.search(r'g\d{1,2}',word)
    if ges is None:
        return True
    else:
        return False

def build_dictionary(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

'''make the list of list of words into list of words'''
words = [item for sublist in training_data for item in sublist]

'''filter the gesture annotations'''
words = [x for x in words if filter_gestures(x)]

'''count the unique appearance of words and assign them a number'''
dictionary, reverse_dictionary = build_dictionary(words)

print(reverse_dictionary[183])
vocab_size = len(dictionary)
print(vocab_size)

any
3220


In [72]:

train_input = []
train_output = []

def create_input_output(sentence):
    
    prev_gesture = False
    
    '''create a 44-length array'''
    single_input = [-1]*int(sentence_length)
    single_output = [0]*(int(sentence_length)+1)
    loc = 0
    sentence_iterator = 0
    
    while loc < int(sentence_length):

        if sentence_iterator < len(sentence):
            word = sentence[sentence_iterator]
            gesture = re.search(r'g\d{1,2}',word)

            if gesture is None:

                prev_gesture = False

                num_word = dictionary[word]

                single_input[loc] = num_word
                loc = loc + 1
                sentence_iterator = sentence_iterator +1

            else:                    
                if not prev_gesture:
                    num_gesture = int(word[1:])
                    del single_input[loc]
                    single_input.append(-1)
                    single_output[loc] = 1
                    prev_gesture = True
                sentence_iterator = sentence_iterator +1
        else:
            break
                
                
    train_input.append(single_input)
    train_output.append(single_output)
    return 'Done!'
        
#create_input_output(training_data[0])
ret_value = [create_input_output(sentence) for sentence in training_data]
print(len(train_input))
print(len(train_input[356]))
print(len(train_output))
print(len(train_output[38]))

print(train_input[38])
print(train_output[38])
print(sentence_length)

874
44
874
45
[235, 124, 5, 15, 582, 25, 1836, 134, 66, 312, 1, 438, 42, 465, 739, 2535, 680, 7, 0, 2412, 680, 235, 120, 95, 638, 9, 71, 26, 2955, 3107, 21, 120, 11, 37, 52, 549, 22, 4, 276, 6, 1892, 69, 183, 6]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
44.0


In [73]:

NUM_EXAMPLES = 800
test_input = np.array(train_input[NUM_EXAMPLES:])
test_output = train_output[NUM_EXAMPLES:] #everything beyond 800
print(len(test_output[0]))
 
train_input = np.array(train_input[:NUM_EXAMPLES])
train_output = train_output[:NUM_EXAMPLES] #till 800


45


In [74]:
import tensorflow as tf
from tensorflow.contrib import rnn

#define constants
#unrolled through 28 time steps
time_steps=11
#hidden LSTM units
num_units=50
#rows of 28 pixels
n_input=4
#learning rate for adam
learning_rate=0.001
#mnist is meant to be classified in 10 classes(0-9).
n_classes=45
#size of batch
batch_size=5

In [75]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

In [76]:
# defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)

with tf.variable_scope("rnn", reuse=None):
    outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")
    #converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
    prediction=tf.matmul(outputs[-1],out_weights)+out_bias

    #loss_function
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    #optimization
    opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    #model evaluationmatmul
    correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [77]:
#initialize variables
batch_init = 0
limit = batch_init + batch_size
init=tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)
    iter=1
    while iter<160:
        
        batch_x = train_input[batch_init: limit]
        batch_y = train_output[batch_init: limit]
        batch_init = batch_init + batch_size
        limit = batch_init + batch_size

        batch_x=batch_x.reshape((batch_size,time_steps,n_input))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %10==0:
            print(outputs[-1])
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1
        
    #calculating test accuracy
    test_data = test_input[:74].reshape((-1, time_steps, n_input))
    test_label = test_output[:74]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))
    #print("Prediction:", sess.run(prediction, feed_dict={x: test_data, y: test_label}))
    #d = prediction.eval(feed_dict={x: test_data}, session=sess)
    #print ("predictions", d)
    #print(type(d[0]))

Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_32:0", shape=(?, 50), dtype=float32)
For iter  10
Accuracy  0.0
Loss  10.6534
__________________
Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_32:0", shape=(?, 50), dtype=float32)
For iter  20
Accuracy  0.0
Loss  6.98304
__________________
Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_32:0", shape=(?, 50), dtype=float32)
For iter  30
Accuracy  0.2
Loss  10.4772
__________________
Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_32:0", shape=(?, 50), dtype=float32)
For iter  40
Accuracy  0.0
Loss  14.4043
__________________
Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_32:0", shape=(?, 50), dtype=float32)
For iter  50
Accuracy  0.2
Loss  8.45381
__________________
Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_32:0", shape=(?, 50), dtype=float32)
For iter  60
Accuracy  0.2
Loss  15.0665
__________________
Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_32:0", shape=(?, 50), dtype=float32)
For iter  70
Accuracy  0.0
Loss  14.5159
__________________
Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_3

In [ ]:
# The code was removed by DSX for sharing.

In [ ]:
# The code was removed by DSX for sharing.